```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
```

```{=latex}
\title{Building Containers for Python Applications}
\author{Moshe Zadka -- https://cobordism.com}
\date{2021}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone

\end{frame}
```

## Good and Bad

* Specifying the requirements
* Reliable builds
* Up to date
* No compilers in prod
* Size
* Support binary wheels
* Not run as root
* Minimal privileges
* Fast rebuilds

## Bases

* Considerations - size
* Considerations - LTS/support
* Considerations - volatility
* Debian
* Ubuntu
* Alpine (probably not)
* CentOS (probably not)
* Rolling releases (probably not)

## Installing Python

## Thinking in Stages

## Use in Applications

## Keeping Up

## Final Thoughts

```{=latex}
\end{document}
```